In [1]:
import os
import shutil
import json
import h5py
import tarfile
from fastai.basics import *
from tqdm.notebook import tqdm
from itertools import islice

WORKING_DIR = '../working'
INPUT_DIR = '../input/traffic4cast2020'
MODEL_DIR = '../input/traffic4cast2020models'
MODEL_NAME = 'istanbul_g091_l07_m.pth'

CITIES = ['ISTANBUL']

# Load test_slots.json

In [2]:
with open(f'{INPUT_DIR}/test_slots.json', 'r') as json_file:
    test_slots = json.load(json_file)
    test_slots = {k:v for each in test_slots for k,v in each.items()}

# Load Model

In [4]:
def ConvLayer(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    '''Conv2d + ELU'''
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                         nn.ELU(inplace=True))

def ConvBlock(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    '''Conv2d + ELU + GroupNorm'''
    return nn.Sequential(ConvLayer(in_channels, out_channels, kernel_size, stride, padding),
                         nn.GroupNorm(num_groups=8, num_channels=out_channels, eps=1e-6))

####################################################################################################

class DenseBlock(nn.Module):
    def __init__(self, in_channels, out_channels, nb_layers=4):
        super().__init__()
        self.first_layer = ConvBlock(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.mid_layers = nn.ModuleList([ConvBlock(in_channels + i*out_channels, out_channels, kernel_size=3, stride=1, padding=1) 
                                         for i in range(1, nb_layers)])
        self.last_layer = ConvLayer(in_channels + nb_layers*out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        layers_concat = list()
        layers_concat.append(x)
        layers_concat.append(self.first_layer(x))
        
        for mid_layer in self.mid_layers:
            layers_concat.append(mid_layer(torch.cat(layers_concat, dim=1)))
            
        return self.last_layer(torch.cat(layers_concat, dim=1))

def AvgPoolDenseBlock(in_channels, out_channels, nb_layers=4):
    return nn.Sequential(nn.AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=True),
                         DenseBlock(in_channels, out_channels, nb_layers))

class UpConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convtrans = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
        self.elu = nn.ELU(inplace=True)
    
    def forward(self, x, x_enc):
        x = self.convtrans(x, output_size=x_enc.shape[-2:])
        x = self.elu(x)
        return torch.cat([x, x_enc], dim=1)

In [5]:
class NetN(nn.Module):
    def __init__(self, num_in_frames=12, num_out_frames=12):
        super().__init__()
        
        self.enc1 = DenseBlock(9*num_in_frames+3*9+7+1, 64, 4)
        self.enc2 = AvgPoolDenseBlock(64, 96, 4)
        self.enc3 = AvgPoolDenseBlock(96, 128, 4)
        self.enc4 = AvgPoolDenseBlock(128, 128, 4)
        self.enc5 = AvgPoolDenseBlock(128, 128, 4)
        self.enc6 = AvgPoolDenseBlock(128, 128, 4)
        self.enc7 = AvgPoolDenseBlock(128, 128, 4)
        self.enc8 = AvgPoolDenseBlock(128, 128, 4)
        
        self.bridge = ConvBlock(128, 128)
        
        self.dec7_1 = UpConvBlock(128, 128)
        self.dec7_2 = ConvBlock(128+128, 128)
        self.dec6_1 = UpConvBlock(128, 128)
        self.dec6_2 = ConvBlock(128+128, 128)
        self.dec5_1 = UpConvBlock(128, 128)
        self.dec5_2 = ConvBlock(128+128, 128)
        self.dec4_1 = UpConvBlock(128, 128)
        self.dec4_2 = ConvBlock(128+128, 128)
        self.dec3_1 = UpConvBlock(128, 128)
        self.dec3_2 = ConvBlock(128+128, 128)
        self.dec2_1 = UpConvBlock(128, 128)
        self.dec2_2 = ConvBlock(128+96, 128)
        self.dec1_1 = UpConvBlock(128, 128)
        self.dec1_2 = ConvBlock(128+64, 128)
        
        self.out_1 = nn.Conv2d(128, 9*num_out_frames, kernel_size=3, stride=1, padding=1)
        self.out_2 = nn.Sigmoid()
          
    def forward(self, x):
        
        x0 = x[:,:108,...]               # 4, 108, 495, 436
        s = x[:,108:115,...]             # 4, 7, 495, 436
        t = x[:,115,...].unsqueeze(1)    # 4, 1, 495, 436
        
        N, C, H, W = x0.shape            # 4, 108, 495, 436
        x0r = x0.reshape(N, 9, 12, H, W) # 4, 9, 12, 495, 436
        x0_mean = x0r.mean(dim=2)        # 4, 9, 495, 436
        x0_std = x0r.std(dim=2)
        x0_rng = x0r.max(dim=2).values - x0r.min(dim=2).values
        
        x = torch.cat([x0, x0_mean, x0_std, x0_rng, s, t], dim=1)
        x = x / 255.
        
        x1 = self.enc1(x)
        x2 = self.enc2(x1)
        x3 = self.enc3(x2)
        x4 = self.enc4(x3)
        x5 = self.enc5(x4)
        x6 = self.enc6(x5)
        x7 = self.enc7(x6)
        x8 = self.enc8(x7)
        
        x100 = self.bridge(x8)
        
        x107 = self.dec7_2(self.dec7_1(x100, x7))
        x106 = self.dec6_2(self.dec6_1(x107, x6))
        x105 = self.dec5_2(self.dec5_1(x106, x5))
        x104 = self.dec4_2(self.dec4_1(x105, x4))
        x103 = self.dec3_2(self.dec3_1(x104, x3))
        x102 = self.dec2_2(self.dec2_1(x103, x2))
        x101 = self.dec1_2(self.dec1_1(x102, x1))
        
        out = self.out_2(self.out_1(x101))*255.
        return out.view(N, 9, -1, H, W)

In [6]:
model = NetN().cuda()
state = torch.load(f'{MODEL_DIR}/{MODEL_NAME}')
hasopt = set(state)=={'model', 'opt'}
model_state = state['model'] if hasopt else state
model.load_state_dict(model_state, strict=True)
model.eval()

NetN(
  (enc1): DenseBlock(
    (first_layer): Sequential(
      (0): Sequential(
        (0): Conv2d(143, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ELU(alpha=1.0, inplace=True)
      )
      (1): GroupNorm(8, 64, eps=1e-06, affine=True)
    )
    (mid_layers): ModuleList(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(207, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ELU(alpha=1.0, inplace=True)
        )
        (1): GroupNorm(8, 64, eps=1e-06, affine=True)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(271, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ELU(alpha=1.0, inplace=True)
        )
        (1): GroupNorm(8, 64, eps=1e-06, affine=True)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(335, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ELU(alpha=1.0, inplace=True)
        )
        (1): GroupNorm(8

# Create Submission

In [7]:
# Clear working directory
!rm -r *

for city in CITIES:
    
    print(city)
    working_path = Path(f'{WORKING_DIR}/{city}')
    
    # Unzip data if it is not already done so
    if not working_path.exists():
        with tarfile.open(f'{INPUT_DIR}/{city}.tar') as tarred_file:
            files = [tarinfo for tarinfo in tarred_file.getmembers()
                     if tarinfo.name.startswith(f'{city}/testing/')
                     or tarinfo.name.startswith(f'{city}/{city}_static')] # Only unzipping the testing folder and static file
            tarred_file.extractall(members=files, path=WORKING_DIR)
    
    # load static features
    with h5py.File(f'{working_path}/{city}_static_2019.h5', 'r') as static_file:
        static_features = static_file.get('array')[()].astype(np.float32)
        static_features = torch.from_numpy(static_features).permute(2, 0, 1)
    static_features = static_features
    
    # Loop through each test date
    for date, frame in tqdm(test_slots.items()): #for date in tqdm(list(islice(test_slots, 3))):
        
        with h5py.File(f'{working_path}/testing/{date}_test.h5', 'r') as h5_file:
            x = h5_file.get('array')[()]
            
        # Note dimension reordering, from (Batch Size, Time, Height, Width, Channels) to (Batch Size, Channels, Time, Height, Width)
        x = np.transpose(x, (0, 4, 1, 2, 3))
        x = torch.from_numpy(x).float()
            
        ##################################################################################################
        # Calculate output
        with torch.no_grad():

            N, C, D, H, W = x.shape
            x = x.reshape(N, C*D, H, W)
            s = torch.stack(N*[static_features], dim=0)
            t = torch.ones(N, 1, H, W)
            for j in range(N):
                t[j] = t[j] * frame[j] * 255. / (288. - 12)
            x = torch.cat([x, s, t], dim=1)
                
            if x.shape[0] > 3:
                y1 = model(x[:3].cuda()).cpu()
                y2 = model(x[3:].cuda()).cpu()
                y = torch.cat([y1, y2])
                del y1, y2
            else:
                y = model(x.cuda()).cpu()
            
            y = torch.round(y)
            y = torch.clamp(y, min=0, max=255)
        ##################################################################################################
        
        # Dimension reordering
        y = y.permute(0, 2, 3, 4, 1).byte()
        # Assume output.shape == input.shape, hence slice out the bit required for submission
        y = y[:,[0,1,2,5,8,11],:,:,:8]
        
        with h5py.File(f'{working_path}/{date}_test.h5', 'w') as h5_file:
            h5_file.create_dataset('array', data=y, compression="gzip", compression_opts=6)
            
        del x, y
        torch.cuda.empty_cache()
        
        # Delete the used files to save disk space...
        os.remove(f'{working_path}/testing/{date}_test.h5')
        
    # Delete data folder
    shutil.rmtree(f'{working_path}/testing')
    os.remove(f'{working_path}/{city}_static_2019.h5')

# Create .zip file
!zip -r0 submission.zip .

ISTANBUL



  adding: __notebook__.ipynb (stored 0%)
  adding: ISTANBUL/ (stored 0%)
  adding: ISTANBUL/2019-07-28_test.h5 (stored 0%)
  adding: ISTANBUL/2019-12-28_test.h5 (stored 0%)
  adding: ISTANBUL/2019-08-04_test.h5 (stored 0%)
  adding: ISTANBUL/2019-09-01_test.h5 (stored 0%)
  adding: ISTANBUL/2019-07-22_test.h5 (stored 0%)
  adding: ISTANBUL/2019-07-30_test.h5 (stored 0%)
  adding: ISTANBUL/2019-12-29_test.h5 (stored 0%)
  adding: ISTANBUL/2019-11-11_test.h5 (stored 0%)
  adding: ISTANBUL/2019-10-24_test.h5 (stored 0%)
  adding: ISTANBUL/2019-12-27_test.h5 (stored 0%)
  adding: ISTANBUL/2019-08-06_test.h5 (stored 0%)
  adding: ISTANBUL/2019-07-02_test.h5 (stored 0%)
  adding: ISTANBUL/2019-08-08_test.h5 (stored 0%)
  adding: ISTANBUL/2019-11-24_test.h5 (stored 0%)
  adding: ISTANBUL/2019-11-12_test.h5 (stored 0%)
  adding: ISTANBUL/2019-09-15_test.h5 (stored 0%)
  adding: ISTANBUL/2019-07-03_test.h5 (stored 0%)
  adding: ISTANBUL/2019-07-13_test.h5 (stored 0%)
  adding: ISTANBUL/2019-10

In [8]:
!zip -d submission *.ipynb

deleting: __notebook__.ipynb
